## Fig Class

In [1]:
import bqplot.pyplot as plt
from bqplot import LinearScale
from traits.api import HasTraits, observe, Any, Array
import numpy as np
import time

dx = 5 # mm
#N2 = 200

N1 = 50
xsc_min = 0
xsc_max = 1000
ysc_min = 0
ysc_max = 500
xsc_margin = 20
ysc_margin = 20

x_acc = np.arange(xsc_min, xsc_max+1e-3, dx)  # Initialize with empty arrays
N2 = len(x_acc)
n_acc = np.array(np.zeros(N2))  # Initialize with empty arrays
z_acc = np.array(np.zeros(N2))  # Initialize with empty arrays


In [2]:
from enum import Enum

class State(Enum):
    IDLE = 1
    WATERING = 2

state = State.IDLE


In [3]:
state == State.IDLE

True

In [4]:

class PlantProfilePlot(HasTraits):

    global N1, N2, xsc_min, ysc_min, xsc_max, ysc_max, xsc_margin, ysc_margin

    fig = plt.figure(layout=dict(height="600px", width="1200px"))
    
    x_scale = LinearScale(min=xsc_min-xsc_margin, max=xsc_max+xsc_margin)  # Set the x-axis limits
    y_scale = LinearScale(min=ysc_min-ysc_margin, max=ysc_max+ysc_margin)  # Set the y-axis limits 

    figdata1 = Array(shape=(N1, 3)) # Col: t, x, z
    figdata2 = Array(shape=(N2, 3))
    
    scatter1 = plt.scatter([], [], colors=['red'], default_size = 30, scales={'x': x_scale, 'y': y_scale})  # Initialize with empty scatter
    scatter2 = plt.scatter([], [], colors=['blue'], default_size = 20, scales={'x': x_scale, 'y': y_scale})  # Initialize with empty scatter

    def __init__(self):
        super(PlantProfilePlot, self).__init__()
        self.figdata1 = np.random.rand(N1, 3)*(1, xsc_max, ysc_max) # Initialize with random data
        self.figdata2 = np.random.rand(N2, 3)*(1, xsc_max, ysc_max) # Initialize with random data

    @observe("figdata1")
    def _on_figdata1_update(self, change):
        with self.scatter1.hold_sync():  # Disable automatic updates during this block
            if self.scatter1.x is not None:
                self.scatter1.x = self.figdata1[:, 1]
                self.scatter1.y = self.figdata1[:, 2]
            else:
                self.scatter1 = plt.scatter(self.figdata1[:, 1], self.figdata1[:, 2], colors=['blue'])

    @observe("figdata2")
    def _on_figdata2_update(self, change):
        with self.scatter2.hold_sync():  # Disable automatic updates during this block
            if self.scatter2.x is not None:
                self.scatter2.x = self.figdata2[:, 1]
                self.scatter2.y = self.figdata2[:, 2]
            else:
                self.scatter2 = plt.scatter(self.figdata2[:, 1], self.figdata2[:, 2], colors=['red'])

    def update_data1(self, new_data):
        self.figdata1 = new_data

    def update_data2(self, new_data):
        self.figdata2 = new_data

    def clear(self):
        self.scatter1.x, self.scatter1.y = [], []
        self.scatter2.x, self.scatter2.y = [], []


plotter = PlantProfilePlot()
plt.show()

In [5]:
# Some test functions ...

# for i in range(100):
#     dat = np.random.rand(N, 3)*(1, 1000, 500)
#     plotter.update_data1(dat)
#     time.sleep(0.1)

# plotter.clear()

# plotter.update_data2(dat)

## Data Structures for Rolling Windows ... 

In [6]:
import numpy as np

In [7]:
x_data = np.linspace(xsc_min, xsc_max, N1)  # Initialize with empty arrays
z_data = np.array(np.zeros(N1))  # Initialize with empty arrays
t_data = np.array(np.zeros(N1))  # Initialize with empty arrays

data = np.array([t_data, x_data, z_data]).T

plotter.update_data1(data)

In [8]:
# n_acc = np.array(np.zeros(N2))  # Initialize with empty arrays
# x_acc = np.array(np.zeros(N2))  # Initialize with empty arrays
# z_acc = np.array(np.zeros(N2))  # Initialize with empty arrays

In [9]:
data2 = np.random.rand(N2, 3)*(1, 1000, 500)
plotter.update_data2(data2)

In [10]:
plotter.clear()

## MQTT

In [11]:
import paho.mqtt.client as mqtt

In [12]:
BROKER = r"kleve.cool"
TOPIC = r"gw/duese002"
TOPIC_OUT = r"gw/duese002-licht"
USER = r"pub"
PW = r"pub"

In [13]:
t = 0
i = 0
x0 = 1200
z0 = 470

In [14]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('HERE THE DECODED MQTT MESSAGES WILL BE SHOWN!\n')
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [15]:
def on_message(client, userdata, message):
    global payload, plotter, out
    global i, t, t0, x, z, traw, xraw, zraw
    global t_data, x_data, z_data
    global n_acc, x_acc, z_acc
    global state

    # Parse the received message
    payload = message.payload.decode('utf-8')
    traw, xraw, zraw = map(float, payload.strip('()').split(','))

    if (i==0): 
        t0 = traw

    t = traw - t0
    x = x0 - xraw
    z = z0 - zraw
    
    s = f"{i:4d}  {t:12.2f}  {x:6.2f}  {z:6.2f}"

    if (state == state.IDLE) & (z > 120):
        client.publish("gw/duese002-licht", "on")
        state = state.WATERING
    if (state == state.WATERING) & (z < 100):
        client.publish("gw/duese002-licht", "off")
        state = state.IDLE
        
    i += 1

    t_data = np.roll(t_data, -1); t_data[-1] = t    
    x_data = np.roll(x_data, -1); x_data[-1] = x
    z_data = np.roll(z_data, -1); z_data[-1] = z

    data = np.array([t_data, x_data, z_data]).T

    plotter.update_data1(data)

# index of x_acc array with the x closest to x_acc[idx]
    idx = int(np.round(x/dx)) 
    if (idx >= 0) & (idx < N2):
        if n_acc[idx] == 0:
            z_acc[idx] = z
        else:
#            z_acc[idx] = (n_acc[idx] * z_acc[idx] + z) / (n_acc[idx] + 1)
            z_acc[idx] = (4 * z_acc[idx] + z) / 5
            
        n_acc[idx] += 1

    data2 = np.array([n_acc, x_acc, z_acc]).T

    plotter.update_data2(data2)
    
#    out.append_stdout(s + "\n")

In [16]:
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    out.append_stdout("Connected with result code " + str(rc) + "\n")

In [17]:
client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

client.username_pw_set(USER, PW)
client.connect(BROKER, 1883, 60)

client.subscribe(TOPIC)

client.loop_start()

In [18]:
#client.publish("TOPIC_OUT", "off")

In [19]:
#client.publish("gw/duese002-licht", "on")